# Assistants

## Introduction

The Assistants API allows you to build AI assistants within your own applications. An Assistant has instructions and can leverage models, tools, and knowledge to respond to user queries. The Assistants API currently supports three types of tools: Code Interpreter, Retrieval, and Function calling.

1. Assistants can call OpenAI’s models with specific instructions to tune their personality and capabilities.
2. Assistants can access multiple tools in parallel. These can be both OpenAI-hosted tools — like Code interpreter and Knowledge retrieval — or tools you build / host (via Function calling).
3. Assistants can access persistent Threads. Threads simplify AI application development by storing message history and truncating it when the conversation gets too long for the model’s context length. You create a Thread once, and simply append Messages to it as your users reply.
4. Assistants can access Files in several formats — either as part of their creation or as part of Threads between Assistants and users. When using tools, Assistants can also create files (e.g., images, spreadsheets, etc) and cite files they reference in the Messages they create.

You can explore the capabilities of the Assistants API using the [Assistants playground](https://platform.openai.com/playground?mode=assistant) or by building a step-by-step integration outlined in this guide. At a high level, a typical integration of the Assistants API has the following flow:

1. Create an Assistant in the API by defining its custom instructions and picking a model. If helpful, enable tools like Code Interpreter, Retrieval, and Function calling.
2. Create a Thread when a user starts a conversation.
3. Add Messages to the Thread as the user ask questions.
4. Run the Assistant on the Thread to trigger responses. This automatically calls the relevant tools.

The Assistants API is in **beta**. Calls to the Assistants API require that you pass a beta HTTP header. This is handled automatically if you’re using OpenAI’s official Python or Node.js SDKs.
```htm
OpenAI-Beta: assistants=v1
```


## How Assistants Work
### Objects
![Objects](https://cdn.openai.com/API/docs/images/diagram-assistant.webp)
| OBJECT   | WHAT IT REPRESENTS |
|----------|--------------------|
| Assistant | Purpose-built AI that uses OpenAI’s models and calls tools |
| Thread   | A conversation session between an Assistant and a user. Threads store Messages and automatically handle truncation to fit content into a model’s context. |
| Message  | A message created by an Assistant or a user. Messages can include text, images, and other files. Messages stored as a list on the Thread. |
| Run      | An invocation of an Assistant on a Thread. The Assistant uses its configuration and the Thread’s Messages to perform tasks by calling models and tools. As part of a Run, the Assistant appends Messages to the Thread. |
| Run Step | A detailed list of steps the Assistant took as part of a Run. An Assistant can call tools or create Messages during its run. Examining Run Steps allows you to introspect how the Assistant is getting to its final results. |


### Step 1: Create an Assistant
An Assistant represents an entity that can be configured to respond to users’ Messages using several parameters like:

* **Instructions**: Use the instructions parameter to guide the personality of the Assistant and define it’s goals. Instructions are similar to system messages in the Chat Completions API.
* **Model**: you can specify any GPT-3.5 or GPT-4 models. The Retrieval tool requires at least `gpt-3.5-turbo-1106` (newer versions are supported) or `gpt-4-turbo-preview` models.
    * Note: Support for fine-tuned models in the Assistants API is coming soon.
* **Tools**: Use the tools parameter to give the Assistant access to up to 128 tools. You can give it access to OpenAI-hosted tools like `code_interpreter` and `retrieval`, or call a third-party tools via a `function` calling.
* **File_ids**: Use the file_ids parameter to give the tools like code_interpreter and retrieval access to files. Files are uploaded using the [File upload endpoint](https://platform.openai.com/docs/api-reference/files/create) and must have the `purpose` set to `assistants` to be used with this API.

In the following example, we're creating an Assistant that is a personal math tutor, with the Code Interpreter tool enabled.


In [2]:
from mlteam_utils import print_object

# In this example, we're creating an Assistant that is a personal math tutor, with the Code Interpreter tool enabled.
from openai import OpenAI
client = OpenAI()

assistant = client.beta.assistants.create(
    name="Math Tutor",
    instructions="You are a personal math tutor. Write and run code to answer math questions.",
    tools=[{"type": "code_interpreter"}],
    model="gpt-4-turbo-preview"
)

### Step 2: Create a Thread
A Thread represents a conversation. We recommend [creating one Thread](https://platform.openai.com/docs/api-reference/threads/createThread) per user as soon as the user initiates the conversation. Pass any user-specific context and files in this thread by [creating Messages](https://platform.openai.com/docs/api-reference/messages/createMessage).

Threads don’t have a size limit. You can add as many Messages as you want to a Thread. The Assistant will ensure that requests to the model fit within the maximum context window, using relevant optimization techniques such as truncation which we have tested extensively with ChatGPT. When you use the Assistants API, you delegate control over how many input tokens are passed to the model for any given Run, this means you have less control over the cost of running your Assistant in some cases but do not have to deal with the complexity of managing the context window yourself.

Organizations that have enabled the [Threads page](https://platform.openai.com/threads) can view Threads created through the Assistants API and Assistants playground. Threads page permissions can be managed in [Organization settings](https://platform.openai.com/account/organization).

In [3]:
thread = client.beta.threads.create()

### Step 3: Add a Message to a Thread
A Message contains text, and optionally any files that you allow the user to upload. Messages need to be added to a specific Thread. Adding images via message objects like in Chat Completions using GPT-4 with Vision is not supported today, but we plan to add support for them in the coming months. You can still upload images and have them processes via retrieval.

In [4]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="I need to solve the equation `3x + 11 = 14`. Can you help me?"
)

# Now if you list the Messages in a Thread, you will see that this message has been appended.
thread_messages = client.beta.threads.messages.list(thread.id)
print_object(thread_messages.data)

[
    {
        "id": "msg_LSe4ZhGdwd5aXwKilhAjDef2",
        "assistant_id": null,
        "content": [
            {
                "text": {
                    "annotations": [],
                    "value": "I need to solve the equation `3x + 11 = 14`. Can you help me?"
                },
                "type": "text"
            }
        ],
        "created_at": 1707206041,
        "file_ids": [],
        "metadata": {},
        "object": "thread.message",
        "role": "user",
        "run_id": null,
        "thread_id": "thread_XPK509FQmEiUVP41k0ADGgN1"
    }
]


### Step 4: Run the Assistant
For the Assistant to respond to the user message, you need to create a Run. This makes the Assistant read the Thread and decide whether to call tools (if they are enabled) or simply use the model to best answer the query. As the run progresses, the assistant appends Messages to the thread with the `role="assistant"`. The Assistant will also automatically decide what previous Messages to include in the context window for the model. This has both an impact on pricing as well as model performance. The current approach has been optimized based on what we learned building ChatGPT and will likely evolve over time.

You can optionally pass new instructions to the Assistant while creating the Run but note that these instructions override the default instructions of the Assistant.

By default, a Run will use the model and tools configuration specified in Assistant object, but you can override most of these when creating the Run for added flexibility.

> ***Note***: file_ids associated with the Assistant cannot be overridden during Run creation. You must use the [modify Assistant](https://platform.openai.com/docs/api-reference/assistants/modifyAssistant) endpoint to do this.

In [5]:
# Trigger the run
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="Please address the user as Jane Doe. The user has a premium account."
)

### Step5: Check the Run Status
Run objects can have multiple statuses.

![Run lifecycle](https://cdn.openai.com/API/docs/images/diagram-1.png)

| STATUS          | DEFINITION |
|-----------------|------------|
| queued          | When Runs are first created or when you complete the required_action, they are moved to a queued status. They should almost immediately move to in_progress. |
| in_progress     | While in_progress, the Assistant uses the model and tools to perform steps. You can view progress being made by the Run by examining the Run Steps. |
| completed       | The Run successfully completed! You can now view all Messages the Assistant added to the Thread, and all the steps the Run took. You can also continue the conversation by adding more user Messages to the Thread and creating another Run. |
| requires_action | When using the Function calling tool, the Run will move to a required_action state once the model determines the names and arguments of the functions to be called. You must then run those functions and submit the outputs before the run proceeds. If the outputs are not provided before the expires_at timestamp passes (roughly 10 mins past creation), the run will move to an expired status. |
| expired         | This happens when the function calling outputs were not submitted before expires_at and the run expires. Additionally, if the runs take too long to execute and go beyond the time stated in expires_at, our systems will expire the run. |
| cancelling      | You can attempt to cancel an in_progress run using the Cancel Run endpoint. Once the attempt to cancel succeeds, status of the Run moves to cancelled. Cancellation is attempted but not guaranteed. |
| cancelled       | Run was successfully cancelled. |
| failed          | You can view the reason for the failure by looking at the last_error object in the Run. The timestamp for the failure will be recorded under failed_at. |

When a Run is in_progress and not in a terminal state, the Thread is locked. This means that:
* New Messages cannot be added to the Thread.
* New Runs cannot be created on the Thread.

In [6]:
# By default, the run goes into the queued state. You can periodically retrieve the Run to check on its status to see if it has moved to completed.
import time

while True:
    time.sleep(5)
    run = client.beta.threads.runs.retrieve(
      thread_id=thread.id,
      run_id=run.id
    )
    if run.status in ['queued', 'in_progress', 'requires_action', 'cancelling']:
        continue
    if run.status in ["completed", "expired", "failed", "cancelled"]:
        print(f"Run is {run.status}")
        break
    break
print_object(run)

Run is completed
{
    "id": "run_zjNVAGjiJF0L3BvDidCfu203",
    "assistant_id": "asst_a1B4jV5omCGjezTJFpukcAw6",
    "cancelled_at": null,
    "completed_at": 1707206069,
    "created_at": 1707206050,
    "expires_at": null,
    "failed_at": null,
    "file_ids": [],
    "instructions": "Please address the user as Jane Doe. The user has a premium account.",
    "last_error": null,
    "metadata": {},
    "model": "gpt-4-turbo-preview",
    "object": "thread.run",
    "required_action": null,
    "started_at": 1707206050,
    "status": "completed",
    "thread_id": "thread_XPK509FQmEiUVP41k0ADGgN1",
    "tools": [
        {
            "type": "code_interpreter"
        }
    ],
    "usage": {
        "completion_tokens": 177,
        "prompt_tokens": 661,
        "total_tokens": 838
    }
}


### Step 6: Display the Assistant's Response
Once the Run completes, you can list the Messages added to the Thread by the Assistant.

In [7]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

#For you to better understand the data structure of the messages
print_object(messages.data)

[
    {
        "id": "msg_jt5PQ3bD7wXXg0WIRte0uP9C",
        "assistant_id": "asst_a1B4jV5omCGjezTJFpukcAw6",
        "content": [
            {
                "text": {
                    "annotations": [],
                    "value": "The solution to the equation \\(3x + 11 = 14\\) is \\(x = 1\\). If you have any more questions or need further assistance, feel free to ask!"
                },
                "type": "text"
            }
        ],
        "created_at": 1707206067,
        "file_ids": [],
        "metadata": {},
        "object": "thread.message",
        "role": "assistant",
        "run_id": "run_zjNVAGjiJF0L3BvDidCfu203",
        "thread_id": "thread_XPK509FQmEiUVP41k0ADGgN1"
    },
    {
        "id": "msg_9p10DE5l40LvuAVGSAan1cjQ",
        "assistant_id": "asst_a1B4jV5omCGjezTJFpukcAw6",
        "content": [
            {
                "text": {
                    "annotations": [],
                    "value": "Sure, we can solve the equation \\(3x + 11 =

In [8]:
# Finally display the messages to the user

# messages.data list holds the last message at the index 0, so we reverse the list to see in chronological order.
for message in reversed(messages.data):
  print(f"_______Role: {message['role']}_______\n")
  # message.content is a list, but I only see 1 item in the list, anyhow used a loop.
  for i, content in enumerate(message['content']):
    print(f"{content['text']['value']}\n")

_______Role: user_______

I need to solve the equation `3x + 11 = 14`. Can you help me?

_______Role: assistant_______

Sure, we can solve the equation \(3x + 11 = 14\) together. To find the value of \(x\), we will follow these steps:

1. Subtract 11 from both sides to isolate the term with \(x\) on one side.
2. Divide by 3 to solve for \(x\).

Let's do the calculation.

_______Role: assistant_______

The solution to the equation \(3x + 11 = 14\) is \(x = 1\). If you have any more questions or need further assistance, feel free to ask!



### Run Steps
![Run steps](https://cdn.openai.com/API/docs/images/diagram-2.png)

Run step statuses have the same meaning as Run statuses.

Most of the interesting detail in the Run Step object lives in the step_details field. There can be two types of step details:
1. `message_creation`: This Run Step is created when the Assistant creates a Message on the Thread.
2. `tool_calls`: This Run Step is created when the Assistant calls a tool. Details around this are covered in the relevant sections of the Tools guide.

In [9]:
run_steps = client.beta.threads.runs.steps.list(
    thread_id=thread.id,
    run_id=run.id
)

for step in run_steps:
    print(f"{step.step_details.type}: {getattr(step.step_details, step.step_details.type)}")

message_creation: MessageCreation(message_id='msg_jt5PQ3bD7wXXg0WIRte0uP9C')
tool_calls: [CodeToolCall(id='call_qe4hURDFkM4CCvihklIryawi', code_interpreter=CodeInterpreter(input="from sympy import symbols, Eq, solve\n\n# Define the variable\nx = symbols('x')\n\n# Define the equation\nequation = Eq(3*x + 11, 14)\n\n# Solve the equation\nsolution = solve(equation, x)\nsolution", outputs=[CodeInterpreterOutputLogs(logs='[1]', type='logs')]), type='code_interpreter')]
message_creation: MessageCreation(message_id='msg_9p10DE5l40LvuAVGSAan1cjQ')


### Cleanup

You can delete your assistant and/or threads. 

In [10]:
# Delete the assistant
client.beta.assistants.delete(assistant_id=assistant.id)

# Delete the thread
client.beta.threads.delete(thread_id=thread.id)

ThreadDeleted(id='thread_XPK509FQmEiUVP41k0ADGgN1', deleted=True, object='thread.deleted')